In [20]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [21]:
# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [22]:
# URL of The amazon Review page
reviews_url = 'https://www.amazon.com/Keyaluo-Solar-Eclipse-Glasses-Approved/dp/B0CTFYT5ZV/ref=pd_rhf_cr_s_pd_crcbs_d_sccl_1_3/144-1773143-5126510?pd_rd_w=grCvx&content-id=amzn1.sym.31346ea4-6dbc-4ac4-b4f3-cbf5f8cab4b9&pf_rd_p=31346ea4-6dbc-4ac4-b4f3-cbf5f8cab4b9&pf_rd_r=GN304VTZSF0SSQTGRJS2&pd_rd_wg=fVkfa&pd_rd_r=b81d8ce2-e7c1-4858-836e-db1c26466f5f&pd_rd_i=B0CTFYT5ZV&th=1'

In [23]:
# Define No of pages
len_page = 5

In [24]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
        }
      
        # Request make for each page
        response = requests.get(url, headers=headers)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [25]:
# Grab Reviews name, description, date from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'
            
        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            content = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            content = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Date' : date,
            'Content' : content
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

In [26]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [27]:
# Empty List to Hold all reviews data
reviews = []

In [28]:
# Iterate all Html page 
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

In [29]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [30]:
df_reviews

,Name,Date,Content
0,Tom Demaree,24/03/2024,User instructions and Warning are straightforw...
1,B. Kirkpatrick,20/03/2024,"Nice quality, proper density for safe solar vi..."
2,Red Napier,27/03/2024,Perfect for eclipse I look at the sun and can ...
3,GJ,18/03/2024,These eclipse glasses are ultra cool. Looking...
4,David Killeen,21/03/2024,Well made but expensive\nRead more
5,TC,21/03/2024,"Good and dark, built very well. They should be..."
6,Barbara,18/03/2024,"Great product , AAS approved\nRead more"
7,paul j.,26/03/2024,received quickly and they look like a high qua...


In [32]:
# Save data
df_reviews.to_csv('reviews.csv', index=False)